In [ ]:
# Choose 3 test functions and compare the implemented algorithms: GA, DE (both versions) and PSO

# 3 test functions
# Each test function will be optimized by each algorithm 11x (to have some sort of reasonable statistic)
# Each test function will be optimized for 5D and 30D. Thus the total number of test functions is 6 (3 functions, each in 2 different dimensional settinsg)
# Stop each run of an algorithm after 3,000*D objective function evaluations (15,000 for 5D and 90,000 for 30D)
# Prepare statistical evaluation - average objective function value (OFV), median, standard deviation, min and max for each algorithm on each function and present the results in a readable format - preferably table (ie. pandas dataframe).

# Extra: Compare the average convegrence of different algorithms on the same function.

In [ ]:
# Import the required libraries
import numpy as np
import pandas as pd

In [ ]:
# Define the test functions
def sphere_function(x):
  return np.sum(x**2)

def schwefel_function(x):
  if isinstance(x, (int, float)):
    x = [x]
  return 418.9829 * len(x) - np.sum(x * np.sin(np.sqrt(np.abs(x))))

def rastrigin_function(x):
  if isinstance(x, (int, float)):
    x = [x]

  return 10 * len(x) + np.sum(np.array(x)**2 - 10 * np.cos(2 * np.pi * np.array(x)))

In [ ]:
# Define the bounds for each functions
optim_funcs= [sphere_function, schwefel_function, rastrigin_function]
optim_space = [[-10,10],[-500, 500],[-5.12, 5.12]]

In [ ]:
# Define the Genetic Algorithm (GA)
def GA(fx, D,generations, population_size=50, elite=20, mutation_probability=0.01,):
    best = None
    history = []

    for _ in range(generations):
        population = np.random.randint(2, size=(population_size, D))
        fit = [fx(individual) for individual in population]
        i_best = np.argmax(fit)
        history.append(fit[i_best])

        if best is None or fit[i_best] > fx(best):
          best = population[i_best]

        if fx(best) == D:
          break

        next_generation = population[:elite]

        while len(next_generation) < population_size:
            parents_indices = np.random.choice(population_size, size=2, replace=False)
            parent1, parent2 = population[parents_indices]
            crossover_point = np.random.randint(D)
            child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
            child1 = np.where(np.random.rand(D) < mutation_probability, 1 - child1, child1)
            child2 = np.where(np.random.rand(D) < mutation_probability, 1 - child2, child2)
            next_generation = np.vstack((next_generation, child1, child2))

        population = next_generation

    return fx(best), history

In [ ]:
# Define the Differential Evolution (Both types rand_1_bin_DE and JDE)
# rand_1_bin_DE
def rand_1_bin_DE(fx, bounds,generations,pop_size=50, CR= 0.3, F=0.5):

  num_dimension = len(bounds)
  population = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(pop_size, num_dimension))

  for generation in range(generations):
    for i in range(pop_size):

      # Mutation
      indices = np.random.choice(pop_size, size=3, replace=False)
      mutant_vector = population[indices[0]] + F * (population[indices[1]] - population[indices[2]])

      # Crossover
      crossover_mask = np.random.rand(num_dimension) < CR
      trial_vector = np.where(crossover_mask, mutant_vector, population[i])

      # Update if better
      trial_fitness = fx(trial_vector)
      current_fitness = fx(population[i])
      if trial_fitness < current_fitness:
        population[i] = trial_vector

  best_solution = population[np.argmin([fx(ind) for ind in population])]
  return best_solution, fx(best_solution)


# JDE
def JDE(fx, bounds,generations, pop_size =50,  tau1 = 0.1, tau2 = 0.1  ):
  num_dimension = len(bounds)

  population = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(pop_size, num_dimension))
  CR = np.random.uniform(0, 1, size=pop_size)
  F = np.random.uniform(0.1, 0.9, size=pop_size)

  for generation in range(generations):
    for i in range(pop_size):

      indices = np.random.choice(pop_size, size=3, replace=False)
      mutant_vector = population[indices[0]] + F[i] * (population[indices[1]] - population[indices[2]])

      # Crossover
      crossover_mask = np.random.rand(num_dimension) < CR[i]
      trial_vector = np.where(crossover_mask, mutant_vector, population[i])


            # Update if better
      trial_fitness = fx(trial_vector)
      current_fitness = fx(population[i])
      if trial_fitness < current_fitness:
          population[i] = trial_vector
          # Update CR and F for individual
          CR[i] = CR[i] * (1 - tau1) + tau1 * np.random.rand()
          F[i] = F[i] * (1 - tau2) + tau2 * np.random.rand()

  best_solution = population[np.argmin([fx(ind) for ind in population])]
  return best_solution, fx(best_solution)

In [ ]:
# Define the Particle Swarm Optimisation (PSO)
# Initialise swarm particles
def init_particles(swarm_size, dims, search_space):
  return np.random.uniform(low=search_space[0], high=search_space[1], size=(swarm_size, dims))

#  Update particle velocity
def update_velocity(velocity,position, pbest, gbest,inert_weight,velocity_range, c1=1.49618, c2=1.49618, ):
  r1 = np.random.rand(*position.shape)
  r2 = np.random.rand(*position.shape)

  cog_component = c1 * r1 * (pbest - position)
  social_component = c2 * r2 * (gbest - position)
  new_velocity = inert_weight * velocity + cog_component + social_component

  # Limiting velocity
  new_velocity = np.clip(new_velocity, -velocity_range, velocity_range)

  return new_velocity

def PSO(fx,fx_space,dims, iterations, swarm_size=30,):
  # c1 = c2 = 1.49618
  inert_weight = 0.9

  # Initialize particles
  particles = init_particles(swarm_size, dims, fx_space)
  velocities = np.zeros_like(particles)

  # Initialize personal best (pbest) positions and values
  pbest = particles.copy()
  pbest_values = np.apply_along_axis(fx, 1, particles)

  # Initialize global best (gbest) position and value
  gbest_index = np.argmin(pbest_values)
  gbest_position = pbest[gbest_index].copy()
  gbest_value = pbest_values[gbest_index]

  # Velocity range calculation
  velocity_range = 0.2 * (fx_space[1] - fx_space[0])


  for iteration in range(iterations):
    # Update inertia weight linearly
    inert_weight = 0.9 - (0.9 - 0.4) * (iteration / iterations)

    # Update velocities and positions
    velocities = update_velocity(velocities, particles, pbest, gbest_position, inert_weight, velocity_range)
    particles += velocities

    # Evaluate new positions
    values = np.apply_along_axis(fx, 1, particles)

    # Update personal best (pbest)
    update_mask = values < pbest_values
    pbest[update_mask] = particles[update_mask].copy()
    pbest_values[update_mask] = values[update_mask]

    # Update global best (gbest)
    gbest_index = np.argmin(pbest_values)
    gbest = pbest[gbest_index].copy()
    gbest_value = pbest_values[gbest_index]

    # Print progress
    # print(f"Iteration {iteration + 1}/{iterations} - Best Value: {gbest_value}")

  return gbest, gbest_value

In [ ]:
# Optimisation Parameters
runs = 11
eval_params = [{"dimensions":5, "iterations": 1500, "type": "5D" },{"dimensions":30, "iterations": 9000, "type": "30D" }]

In [ ]:
# Define the optimization experiment
def optimisation_experiment(optim_funcs=optim_funcs, eval_params=eval_params, runs= runs, ):
  GA_results = []
  Rand_DE_results = []
  JDE_results = []
  PSO_results = []

  for run in range(runs):
      for a , eval_param in enumerate(eval_params):
        for b , fx in enumerate(optim_funcs):
          bounds = np.array([optim_space[b]] * eval_param['dimensions'])
          ga_best, ga_history = GA(fx, eval_param['dimensions'], eval_param['iterations'])
          rand_solution, rand_best = rand_1_bin_DE(fx, bounds,eval_param['iterations'] )
          jde_solution, jde_best =  JDE(fx, bounds,eval_param['iterations'] )
          pso_solution, pso_best = PSO(fx,optim_space[b], eval_param['dimensions'],eval_param['iterations'] )

          GA_results.append({
            "Optimisation Function": fx.__name__,
            'Run': run + 1,
            'Objective Function Value': ga_best,
            'Best Solution': ga_history,
            'Type': eval_param['type']
          })

          Rand_DE_results.append({
            "Optimisation Function": fx.__name__,
            'Run': run + 1,
            'Objective Function Value': rand_best,
            'Best Solution': rand_solution,
            'Type': eval_param['type']
          })

          JDE_results.append({
            "Optimisation Function": fx.__name__,
            'Run': run + 1,
            'Objective Function Value': jde_best,
            'Best Solution': jde_solution,
            'Type': eval_param['type']
          })

          PSO_results.append({
            "Optimisation Function": fx.__name__,
            'Run': run + 1,
            'Objective Function Value': pso_best,
            'Best Solution': pso_solution,
            'Type': eval_param['type']
          })



  return GA_results, Rand_DE_results, JDE_results, PSO_results

ga, rand_de, jde, pso = optimisation_experiment(optim_funcs)

In [ ]:
# Print the results
def print_results(result):
  for i, result in enumerate(result):
    print(f"\nRun {result['Run']} --- {result['Optimisation Function']}")
    print(f"Type - {result['Type']}")
    print(f"Best Value {result['Objective Function Value']}")
    print(f"Best Solution {result['Best Solution']}")

print_results(ga)

print_results(rand_de)

print_results(jde)

print(pso)


Run 1 --- sphere_function
Type - 5D
Best Value 5
Best Solution [5]

Run 1 --- schwefel_function
Type - 5D
Best Value 2094.9145
Best Solution [2094.073029015192, 2094.073029015192, 2094.9145, 2094.9145, 2094.9145, 2094.9145, 2094.9145, 2094.9145, 2094.9145, 2094.9145, 2094.9145, 2094.9145, 2094.9145, 2094.9145, 2094.9145, 2094.9145, 2094.9145, 2094.9145, 2094.9145, 2094.9145, 2094.9145, 2094.9145, 2094.073029015192, 2094.073029015192, 2094.9145, 2094.073029015192, 2094.9145, 2094.9145, 2094.073029015192, 2094.9145, 2094.9145, 2094.9145, 2094.9145, 2094.073029015192, 2094.9145, 2094.9145, 2094.9145, 2094.9145, 2094.9145, 2094.9145, 2094.9145, 2094.073029015192, 2094.9145, 2094.073029015192, 2094.073029015192, 2094.9145, 2094.9145, 2094.9145, 2094.9145, 2094.073029015192, 2094.9145, 2094.9145, 2094.073029015192, 2094.073029015192, 2094.9145, 2094.9145, 2094.073029015192, 2094.9145, 2094.9145, 2094.9145, 2094.073029015192, 2094.9145, 2094.9145, 2094.073029015192, 2094.073029015192, 2094.0

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# Convert to Dataframes

df_GA =  pd.DataFrame(ga)
df_Rand_DE =  pd.DataFrame(rand_de)
df_JDE =  pd.DataFrame(jde)
df_PSO =  pd.DataFrame(pso)

In [ ]:
# Statistical evaluation for Genetic Algorithm
def statistical_evaluation(df):
    average_fitness = df['Objective Function Value'].mean()
    median_fitness = df['Objective Function Value'].median()
    std_dev_fitness = df['Objective Function Value'].std()
    min_fitness = df['Objective Function Value'].min()
    max_fitness = df['Objective Function Value'].max()


    return average_fitness, median_fitness, std_dev_fitness, min_fitness, max_fitness

In [ ]:
# Define the algorithms and optimization functions
algorithms = ['GA', 'Rand_DE', 'JDE', 'PSO']
optim_funcs = ['sphere_function', 'schwefel_function', 'rastrigin_function']

# Create a dictionary to store the dataframes for each algorithm
dfs = {'GA': df_GA, 'Rand_DE': df_Rand_DE, 'JDE': df_JDE, 'PSO': df_PSO}

# Loop over each algorithm
for algo in algorithms:
    # Loop over each optimization function
    for func in optim_funcs:
        # Calculate the statistics
        avg, median, std, min_val, max_val = statistical_evaluation(dfs[algo].loc[dfs[algo]['Optimisation Function'] == func])

        # Print the results
        print(f"{algo} - {func}")
        print(f" Average: {avg}" )
        print(f" Median: {median}" )
        print(f" Standard Deviation: {std}" )
        print(f" Min: {min_val}" )
        print(f" Max: {max_val} \n" )

GA - sphere_function
 Average: 16.181818181818183
 Median: 16.0
 Standard Deviation: 11.454408025012173
 Min: 5.0
 Max: 29.0 

GA - schwefel_function
 Average: 7330.862046160532
 Median: 7330.517808030383
 Standard Deviation: 5359.16318143399
 Min: 2094.9145
 Max: 12567.804058030384 

GA - rastrigin_function
 Average: 15.954545454545455
 Median: 15.5
 Standard Deviation: 11.214265034309658
 Min: 5.0
 Max: 27.0 

Rand_DE - sphere_function
 Average: 5.464010714041011e-107
 Median: 2.2101849945580747e-110
 Standard Deviation: 2.213874406170317e-106
 Min: 3.5388696739775976e-146
 Max: 1.0382578728314847e-105 

Rand_DE - schwefel_function
 Average: -9.32205288305796e+69
 Median: -5.423317528840958e+59
 Standard Deviation: 3.9088869359428614e+70
 Min: -1.8355524676275042e+71
 Max: -89727503790.18675 

Rand_DE - rastrigin_function
 Average: 0.8469190406844818
 Median: 0.0
 Standard Deviation: 2.7434809473791026
 Min: 0.0
 Max: 9.6776913714761 

JDE - sphere_function
 Average: 2.63509534245161

In [ ]:
# Compare the average convergence of different algorithms on the same function
for func in optim_funcs:
    print(f"\nAverage Convergence for {func}")
    for algo in algorithms:
        avg_convergence = dfs[algo].loc[dfs[algo]['Optimisation Function'] == func, 'Objective Function Value'].mean()
        print(f"{algo}: {avg_convergence}")


Average Convergence for sphere_function
GA: 16.181818181818183
Rand_DE: 5.464010714041011e-107
JDE: 2.6350953424516102e-132
PSO: 120.72494888881788

Average Convergence for schwefel_function
GA: 7330.862046160532
Rand_DE: -9.32205288305796e+69
JDE: -1.6804162626616954e+107
PSO: 3248.538902286163

Average Convergence for rastrigin_function
GA: 15.954545454545455
Rand_DE: 0.8469190406844818
JDE: 37.030719168231045
PSO: 138.7493670549459
